## Overview

This notebook will show SCD types 1 2 & 3 implementation using the delta framework

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, sha2, udf, array
from delta.tables import *
from datetime import datetime, timedelta

In [0]:
dbutils.fs.rm("/tmp/bronze", True)
dbutils.fs.rm("/tmp/silver", True)

Out[2]: True

In [0]:
%sql

DROP TABLE IF EXISTS customer_change_bronze;
DROP TABLE IF EXISTS customer_silver_scd1;
DROP TABLE IF EXISTS customer_silver_scd2;
DROP TABLE IF EXISTS customer_silver_scd3;

In [0]:
customerJSON = ' [                                                                            \
	{                                                                                         \
		"id": "1",                                                                            \
		"name": "Julie Rich",                                                                 \
		"address": "Ap #255-3031 Dui Avenue, Billings, USA",                                  \
		"phone": "1-528-884-4331",                                                            \
		"ingestdate": "2017-05-12"                                                            \
	},                                                                                        \
    {                                                                                         \
		"id": "2",                                                                            \
		"name": "Eden Byrd",                                                                  \
		"address": "Ap #476-7527 Aenean Road, Harrisburg, USA",                               \
		"phone": "1-375-910-8385",                                                            \
		"ingestdate": "2020-04-19"                                                            \
	},                                                                                        \
    {                                                                                         \
		"id": "3",                                                                            \
		"name": "Magee Cash",                                                                 \
		"address": "P.O. Box 333, 5820 Vel Avenue, New Haven, USA",                           \
		"phone": "1-853-137-7417",                                                            \
		"ingestdate": "2019-02-22"                                                            \
	}                                                                                         \
 ]'

customerSCD1DF = spark.read.json(sc.parallelize([customerJSON]))
customerSCD1DF.write.format("delta").option("path", "dbfs:/tmp/silver/customer_silver_scd1").saveAsTable("customer_silver_scd1")
display(customerSCD1DF.select("ID", "name", "address", "phone", "ingestdate"))

ID,name,address,phone,ingestdate
1,Julie Rich,"Ap #255-3031 Dui Avenue, Billings, USA",1-528-884-4331,2017-05-12
2,Eden Byrd,"Ap #476-7527 Aenean Road, Harrisburg, USA",1-375-910-8385,2020-04-19
3,Magee Cash,"P.O. Box 333, 5820 Vel Avenue, New Haven, USA",1-853-137-7417,2019-02-22


In [0]:
customerChangeJSON = '[                                                                       \
	 {                                                                                        \
		"id": "3",                                                                            \
		"name": "Magee Cash",                                                                 \
		"address": "687 Magnis Rd., New Haven, USA",                                          \
		"phone": "1-853-137-7417",                                                            \
		"ingestdate": "2022-03-24"                                                            \
	}                                                                                         \
]'
customerChangeDF = spark.read.json(sc.parallelize([customerChangeJSON]))
customerChangeDF.write.format("delta").mode("overwrite").option("path", "dbfs:/tmp/bronze/customer_change").saveAsTable("customer_change_bronze")
display(customerChangeDF.select("ID", "name", "address", "phone", "ingestdate"))

ID,name,address,phone,ingestdate
3,Magee Cash,"687 Magnis Rd., New Haven, USA",1-853-137-7417,2022-03-24


## SCD TYPE 1

In [0]:
%sql
MERGE INTO customer_silver_scd1 c
    USING customer_change_bronze cc
    ON c.id = cc.id
    WHEN MATCHED 
        THEN UPDATE SET address          = cc.address,
                        name             = cc.name,
                        phone            = cc.phone
    WHEN NOT MATCHED 
        THEN INSERT *


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
%sql

select version, operation, operationParameters, operationMetrics from
(describe history customer_silver_scd1)

version,operation,operationParameters,operationMetrics
1,MERGE,"Map(predicate -> (c.`id` = cc.`id`), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])","Map(numTargetRowsCopied -> 2, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 13406, numTargetRowsInserted -> 0, scanTimeMs -> 9940, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 1, rewriteTimeMs -> 3345)"
0,CREATE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","Map(numFiles -> 1, numOutputBytes -> 1746, numOutputRows -> 3)"


In [0]:
%sql
SELECT id, name, address, phone from customer_silver_scd1

## SCD TYPE 2

In [0]:
dbutils.fs.rm("/tmp/silver/customer_silver_scd2", True)

In [0]:
%sql
DROP TABLE IF EXISTS customer_silver_scd2;


In [0]:
customerSCD2DF = spark.read.json(sc.parallelize([customerJSON]))
customerSCD2DF = customerSCD2DF.withColumn('enddate', lit(''))    \
                               .withColumn('status', lit('Current')) \
                               .withColumnRenamed('ingestdate', 'startdate') 
customerSCD2DF.write.format("delta").option("path", "dbfs:/tmp/silver/customer_silver_scd2").saveAsTable("customer_silver_scd2")
display(customerSCD2DF.select("ID", "name", "address", "phone", "startdate", "enddate", "status"))


In [0]:
%sql

MERGE INTO customer_silver_scd2 AS c
    USING
    (
    SELECT cc.id AS mergerkey, 
           cc.id, cc.name, cc.phone, cc.address, cc.ingestdate as startdate,
           '' as enddate,
           'Current' as status
       FROM customer_change_bronze cc
    UNION ALL
    SELECT NULL as mergerkey, 
           cc.id, cc.name, cc.phone, cc.address, cc.ingestdate as startdate,
           '' as enddate,
           'Current' as status
       FROM customer_change_bronze cc
    ) ud
    ON c.id = ud.mergerkey
    AND c.status = 'Current'
    WHEN MATCHED 
        THEN UPDATE SET status           = 'Expired',
                        enddate          = ud.startdate
    WHEN NOT MATCHED 
    AND mergerkey is null 
        THEN INSERT *

In [0]:
%sql
select id, name, address, phone, startdate, enddate, status from customer_silver_scd2 order by id

## SCD TYPE 3

In [0]:
dbutils.fs.rm("/tmp/silver/customer_silver_scd3", True)

In [0]:
%sql
DROP TABLE IF EXISTS customer_silver_scd3;


In [0]:
customerSCD3DF = spark.read.json(sc.parallelize([customerJSON]))
customerSCD3DF = customerSCD3DF.withColumnRenamed('ingestdate', 'modifieddate') \
                               .withColumn('previous_address', lit('')) \
                               .withColumn('previous_name', lit('')) \
                               .withColumn('previous_phone', lit(''))
customerSCD3DF.write.format("delta").option("path", "dbfs:/tmp/silver/customer_silver_scd3").saveAsTable("customer_silver_scd3")
display(customerSCD3DF.select("ID", "name", "previous_name", "address", "previous_address",  "phone",  "previous_phone", "modifieddate"))

In [0]:
%sql
MERGE INTO customer_silver_scd3 c
    USING customer_change_bronze cc
    ON c.id = cc.id
    WHEN MATCHED THEN UPDATE SET c.previous_address = c.address,
                                 c.address          = cc.address,
                                 c.modifieddate     = current_date()
    WHEN NOT MATCHED THEN INSERT (ID,                               
                                  name,
                                  previous_name,
                                  address,
                                  previous_address,
                                  phone,
                                  previous_phone, 
                                  modifieddate)
                     VALUES      (cc.ID,
                                  cc.name,
                                  '',
                                  cc.address,
                                  '',
                                  cc.phone,
                                  '',
                                  current_date())             

In [0]:
%sql
select id, name, previous_name, address, previous_address, phone, previous_phone, modifieddate from customer_silver_scd3